## 实现多元线性回归模型

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

### 1. 切分训练数据集和测试数据集

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

### 2. 使用我们自己的LinearRegression

In [6]:
%run D:\\python-code\LinearRegression.py

In [7]:
reg = LinearRegression()
reg.fit_normal(X_train, y_train)

LinearRegression()

In [8]:
reg.coef_  # 系数

array([-1.18919477e-01,  3.63991462e-02, -3.56494193e-02,  5.66737830e-02,
       -1.16195486e+01,  3.42022185e+00, -2.31470282e-02, -1.19509560e+00,
        2.59339091e-01, -1.40112724e-02, -8.36521175e-01,  7.92283639e-03,
       -3.81966137e-01])

In [9]:
reg.interception_  # 截距

34.16143549621691

因为现在已经是多维，所以我们已经不能进行可视化处理了

In [10]:
reg.score(X_test, y_test) # R方

0.8129802602658533

简单线性回归中只用一个特征预测的R方为0.6左右，使用了多个特征值，R方达到了0.8。从某种程度上也印证了如果我们的数据它的特征更多的话，并且这些特征如果真的能够非常好的反映我们最终要预测的那个指标，在这里就是房价那个指标的话，那么相应的，使用更多的特征这样的数据最终的预测结果会是更好的。

## 使用 sklearn 中多元线性回归模型

In [12]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

In [13]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [14]:
lin_reg.coef_ # 系数

array([-1.18919477e-01,  3.63991462e-02, -3.56494193e-02,  5.66737830e-02,
       -1.16195486e+01,  3.42022185e+00, -2.31470282e-02, -1.19509560e+00,
        2.59339091e-01, -1.40112724e-02, -8.36521175e-01,  7.92283639e-03,
       -3.81966137e-01])

In [15]:
lin_reg.intercept_ # 截距

34.16143549624624

In [16]:
lin_reg.score(X_test, y_test)

0.8129802602658492

## 使用 kNN 算法解决回归问题

### kNN Regressor

In [17]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor() # 默认k=5
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test)

0.5865412198300899

从结果可以看出，kNN 算法的准确性只有 0.58，是远远低于线性回归的，但是不要忘记了，kNN 算法中存在很多的超参数，下面我们就用网格搜索来实验一下最好的超参数

In [20]:
from sklearn.grid_search import GridSearchCV
param_grid = [
    {
        'weights': ['uniform'],
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    }
]

In [25]:
knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    2.0s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [26]:
grid_search.best_params_

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [27]:
grid_search.best_score_

0.634093080186858

我们发现0.6还是比0.8差远了。但是这里的best_score_方法与我们之前的 score 方法求解方式不一样，这是因为网格搜索使用了交叉验证（CV）的方式。

那么为了得到和前面数据同样衡量标准，我们应该使用下面这种方法。

In [28]:
grid_search.best_estimator_.score(X_test, y_test)

0.7044357727037996

最终得到的结果为0.7，这个值才是真正的和我们前面所得到的结果在同一个衡量标准下的相应的值。

我们发现0.7要比使用kNN默认的参数所得到的0.5要好，但是它是不如我们使用线性回归算法得到的结果的。但是，这里需要注意，其实有一部分原因在于我们真正使用网格搜索的时候，我们在搜索的过程中比较的score函数是那个GridSeachCV里面的score的计算方法，换句话说，我们没有跳出来使用我们的score的计算方法在这些参数中搜索获得到的那组最佳的参数，这一点需要注意。

通过这个例子，其实也想告诉大家。我们在使用机器学习算法的过程中，会用各种方法得到各种不同的评价标准，我们在比较这些评价标准的时候，一定要非常小心，很多时候可能我们不能非常武断的直接说通过我们这样的工作流程得到的这种算法比这种算法更加的好，这里就是一个很好的例子。

In [32]:
%%time
best_method = ""
best_p = -1
best_score = 0.0
best_k = -1
for method in ["uniform", "distance"]:
    for k in range(1, 11):
        knn_reg = KNeighborsRegressor(n_neighbors=k, weights=method, p=p)
        knn_reg.fit(X_train, y_train)
        score = knn_reg.score(X_test, y_test)
        if(score > best_score):
            best_k = k
            best_p = p
            best_method = method
            best_score = score

print("best_k = ", best_k)
print("best_p = ", best_p)
print("best_method = ", best_method)
print("best_score = ", best_score)
            

best_k =  9
best_p =  1
best_method =  distance
best_score =  0.7287667475661718
Wall time: 17.2 ms


In [33]:
knn_reg = KNeighborsRegressor(n_neighbors=9, weights="distance", p=1)
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test)

0.7287667475661718

我们发现使用for循环搜索出来的超参数是和使用网格搜索搜索出来的超参数是不同的，而且得到的准确率也要比网格搜索得到的准确率要高一些，但是还是低于线性回归算法。